### Soft Data Integration with Indicator Kriging 

#### Michael Pyrcz, Professor, The University of Texas at Austin 


#### Contacts: [Twitter/@GeostatsGuy](https://twitter.com/geostatsguy) | [GitHub/GeostatsGuy](https://github.com/GeostatsGuy) | [www.michaelpyrcz.com](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446)

#### Load the required libraries

The following code loads the required libraries.

In [1]:
supress_warnings = False

import os                                                 # to set current working directory 
import numpy as np                                        # arrays and matrix math
import pandas as pd                                       # DataFrames
import matplotlib.pyplot as plt                           # plotting
from scipy.interpolate import make_interp_spline          # smooth curves
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks

import geostatspy.geostats as geostats
import geostatspy.GSLIB as GSLIB

plt.rc('axes', axisbelow=True)                            # grid behind plotting elements
if supress_warnings == True:
    import warnings                                       # supress any warnings for this demonstration
    warnings.filterwarnings('ignore')  

#### Set the working directory

I always like to do this so I don't lose files and to simplify subsequent read and writes (avoid including the full address each time). 

In [2]:
os.chdir(r"C:\Users\17137\OneDrive - The University of Texas at Austin\Courses\DataSets")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\17137\\OneDrive - The University of Texas at Austin\\Courses\\DataSets'

#### Load Data

Here's the command to load our comma delimited data file in to a Pandas' DataFrame object.  For fun try misspelling the name. You will get an ugly, long error. 

In [ ]:
#df = pd.read_csv('https://raw.githubusercontent.com/GeostatsGuy/GeoDataSets/master/SoftData_Contact2D.csv')     # load our data table 
df = pd.read_csv('SoftData_Contact2D.csv')     # load our data table (wrong name!)

We loaded our file into our DataFrame called 'df'. But how do you really know that it worked? Visualizing the DataFrame would be useful and we already leard about these methods in this demo (https://git.io/fNgRW). 

We can preview the DataFrame by printing a slice or by utilizing the 'head' DataFrame member function (with a nice and clean format, see below). With the slice we could look at any subset of the data table and with the head command, add parameter 'n=13' to see the first 13 rows of the dataset.  

In [ ]:
df.head(n=13)                           # we could also use this command for a table preview

In [ ]:
# xmin = 0.0; xmax = 1000.0               # range of x values
# ymin = 0.0; ymax = 1000.0               # range of y values

# xsiz = 10; ysiz = 10
# nx = 100; ny = 100
# xmn = 5; ymn = 5

# cmap = plt.cm.plasma                    # color map
# GSLIB.locmap(df,'X','Y','Facies',xmin,xmax,ymin,ymax,0,1,'Well Data - Facies','X(m)','Y(m)','Facies (0 - shale, 1 - sand)',cmap,'locmap_facies')

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  

#### Declare functions

These are some of the functions from GeostatsPy required by the new program.  We will declare them here and then in the future integrate the new indicator kriging program into the package properly.

In [ ]:
def setup_rotmat(c0, nst, it, cc, ang, pmx):

    PI = 3.141_592_65
    DTOR = PI / 180.0

    # The first time around, re-initialize the cosine matrix for the variogram
    # structures
    rotmat = np.zeros((4, nst))
    maxcov = c0
    for js in range(0, nst):
        azmuth = (90.0 - ang[js]) * DTOR
        rotmat[0, js] = math.cos(azmuth)
        rotmat[1, js] = math.sin(azmuth)
        rotmat[2, js] = -1 * math.sin(azmuth)
        rotmat[3, js] = math.cos(azmuth)
        if it[js] == 4:
            maxcov = maxcov + pmx
        else:
            maxcov = maxcov + cc[js]
    return rotmat, maxcov

def ksol_numpy(neq, a, r):
    a = a[0: neq * neq]  # trim the array
    a = np.reshape(a, (neq, neq))  # reshape to 2D
    ainv = linalg.inv(a)  # invert matrix
    r = r[0: neq]  # trim the array
    s = np.matmul(ainv, r)  # matrix multiplication
    return s

def cova2(x1, y1, x2, y2, nst, c0, pmx, cc, aa, it, ang, anis, rotmat, maxcov):
    EPSLON = 0.000_000
    # Check for very small distance
    dx = x2 - x1
    dy = y2 - y1
    if (dx * dx + dy * dy) < EPSLON:
        cova2_ = maxcov
        return cova2_

    # Non-zero distance, loop over all the structures
    cova2_ = 0.0
    for js in range(0, nst):
        # Compute the appropriate structural distance
        dx1 = dx * rotmat[0, js] + dy * rotmat[1, js]
        dy1 = (dx * rotmat[2, js] + dy * rotmat[3, js]) / anis[js]
        h = math.sqrt(max((dx1 * dx1 + dy1 * dy1), 0.0))
#        print('cova h'); print(h)
        if it[js] == 1:
            # Spherical model
            hr = h / aa[js]
            if hr < 1.0:
                cova2_ = cova2_ + cc[js] * (1.0 - hr * (1.5 - 0.5 * hr * hr))
        elif it[js] == 2:
            # Exponential model
            cova2_ = cova2_ + cc[js] * np.exp(-3.0 * h / aa[js])
        elif it[js] == 3:
            # Gaussian model
            hh = -3.0 * (h * h) / (aa[js] * aa[js])
            cova2_ = cova2_ + cc[js] * np.exp(hh)
        elif it[js] == 4:
            # Power model
            cov1 = pmx - cc[js] * (h ** aa[js])
            cova2_ = cova2_ + cov1
    return cova2_

def add_grid():
    plt.gca().grid(True, which='major',linewidth = 1.0); plt.gca().grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks   

Here is the new indicator kriging program along with a couple of supporting functions.

In [ ]:
import scipy.spatial as sp         # for fast nearest nearbour search
import math  # for trig functions etc.
import numpy.linalg as linalg  # for linear algebra

def correct_trend(trend):
    ny = trend.shape[0]
    nx = trend.shape[1]
    ncut = trend.shape[2]
    for iy in range(0,ny):
        for ix in range(0,nx):
            sum = 0.0
            for ic in range(0,ncut):
                sum = sum + trend[iy,ix,ic]
            if sum > 0.0:
                for icut in range(0,ncut):
                    trend[iy,ix,ic] = trend[iy,ix,ic] / sum
    return trend

def ordrel(ivtype,ncut,ccdf):
#    print('input ordering relations'); print(ccdf)
    ccdfo = np.zeros(ncut)
    print('Inside ordel' + str(ccdf))
    ccdf1 = np.zeros(ncut)
    ccdf2 = np.zeros(ncut) # do we need MAXCUT = 100 for these 2?

# Make sure conditional cdf is within [0,1]:
    for i in range(0,ncut):
        if ccdf[i] < 0.0:
            ccdf1[i] = 0.0
            ccdf2[i] = 0.0
        elif ccdf[i] > 1.0:
            ccdf1[i] = 1.0
            ccdf2[i] = 1.0
        else:
            ccdf1[i] = ccdf[i]
            ccdf2[i] = ccdf[i]
#    print('ordering relations'); print(ccdf1,ccdf2)

# Correct sequentially up, then down, and then average:
    if ivtype == 0:
        sumcdf = 0.0
        for i in range(0,ncut):
            sumcdf = sumcdf + ccdf1[i]
        if sumcdf <= 0.0: sumcdf = 1.0
        for i in range(0,ncut):
            ccdfo[i] = ccdf1[i] / sumcdf
    else:
        for i in range(1,ncut):
            if ccdf1[i] < ccdf1[i-1]: ccdf1[i] = ccdf1[i-1]
        for i in range(ncut-2,0,-1):
            if ccdf2[i] > ccdf2[i+1]: ccdf2[i] = ccdf2[i+1]
        for i in range(0,ncut):
            ccdfo[i] = 0.5*(ccdf1[i]+ccdf2[i])

    print('ccdf1' + str(ccdf1))
    print('ccdf2' + str(ccdf2))
    print('ccdfo' + str(ccdfo))
            
# Return with corrected CDF:
    return ccdfo

def ik2d(df,xcol,ycol,vcol,readind,colstart,ivtype,koption,ncut,thresh,gcdf,trend,tmin,tmax,nx,xmn,xsiz,ny,ymn,ysiz,
         ndmin,ndmax,radius,ktype,vario): 

# Find the needed paramters:
    PMX = 9999.9
    MAXSAM = ndmax + 1
    MAXEQ = MAXSAM + 1
    mik = 0  # full indicator kriging
    use_trend = False
    if trend.shape[0] == nx and trend.shape[1] == ny and trend.shape[2] == ncut: use_trend = True
    
# load the variogram
    MAXNST = 2
    nst = np.zeros(ncut,dtype=int); c0 = np.zeros(ncut); cc = np.zeros((MAXNST,ncut)) 
    aa = np.zeros((MAXNST,ncut),dtype=int); it = np.zeros((MAXNST,ncut),dtype=int) 
    ang = np.zeros((MAXNST,ncut)); anis = np.zeros((MAXNST,ncut))

    for icut in range(0,ncut):
        nst[icut] = int(vario[icut]['nst'])
        c0[icut] = vario[icut]['nug']; cc[0,icut] = vario[icut]['cc1']; it[0,icut] = vario[icut]['it1']; 
        ang[0,icut] = vario[icut]['azi1']; 
        aa[0,icut] = vario[icut]['hmaj1']; anis[0,icut] = vario[icut]['hmin1']/vario[icut]['hmaj1'];
        if nst[icut] == 2:
            cc[1,icut] = vario[icut]['cc2']; it[1,icut] = vario[icut]['it2']; ang[1,icut] = vario[icut]['azi2']; 
            aa[1,icut] = vario[icut]['hmaj2']; anis[1,icut] = vario[icut]['hmin2']/vario[icut]['hmaj2'];

# Load the data
    if readind is True:
        df_extract = df
    else:
        df_extract = df.loc[(df[vcol] >= tmin) & (df[vcol] <= tmax)]    # trim values outside tmin and tmax
    MAXDAT = len(df_extract)
    MAXCUT = ncut
    MAXNST = 2
    MAXROT = MAXNST*MAXCUT+ 1
    ikout = np.zeros((ny,nx,ncut))
    maxcov = np.zeros(ncut)
            
    # Allocate the needed memory:   
    xa = np.zeros(MAXSAM)
    ya = np.zeros(MAXSAM)
    vra = np.zeros(MAXSAM)
    dist = np.zeros(MAXSAM)
    nums = np.zeros(MAXSAM)
    r = np.zeros(MAXEQ)
    rr = np.zeros(MAXEQ)
    s = np.zeros(MAXEQ)
    a = np.zeros(MAXEQ*MAXEQ)
    print('Grid set up' + str(ny) + ' ' + str(nx))
    vr = np.zeros((MAXDAT,MAXCUT+1))
    print("MAXCUT+1 " + str(MAXCUT+1))
    
    nviol = np.zeros(MAXCUT)
    aviol = np.zeros(MAXCUT)
    xviol = np.zeros(MAXCUT)
    
    ccdf = np.zeros(ncut)
    ccdfo = np.zeros(ncut)
    ikout = np.zeros((ny,nx,ncut))
    
    x = df_extract[xcol].values
    y = df_extract[ycol].values
    v = df_extract[vcol].values
    print('Data values ' + str(v))
    
# The indicator data are constructed knowing the thresholds and the
# data value.

    if readind == True:
        for idata in range(0,len(df)):
            for icut in range(0,ncut):
                print(icut+colstart)
                vr[idata,icut] = df.iloc[idata,icut+colstart]
                
        print('Read in soft data:')

    else:
        if ivtype == 0:
            for icut in range(0,ncut): 
                vr[:,icut] = np.where((v <= thresh[icut] + 0.5) & (v > thresh[icut] - 0.5), '1', '0')
        else:
            for icut in range(0,ncut): 
                vr[:,icut] = np.where(v <= thresh[icut], '1', '0')
    
    vr[:,ncut] = v
    print(vr)
    
# Make a KDTree for fast search of nearest neighbours   
    dp = list((y[i], x[i]) for i in range(0,MAXDAT))
    data_locs = np.column_stack((y,x))
    tree = sp.cKDTree(data_locs, leafsize=16, compact_nodes=True, copy_data=False, balanced_tree=True)
    
# Summary statistics of the input data
    
    avg = vr[:,ncut].mean()
    stdev = vr[:,ncut].std()
    ss = stdev**2.0
    vrmin = vr[:,ncut].min()
    vrmax = vr[:,ncut].max()
    print('Data for IK3D: Variable column ' + str(vcol))
    print('  Number   = ' + str(MAXDAT))
    ndh = MAXDAT
    
    actloc = np.zeros(MAXDAT, dtype = int)
    for i in range(1,MAXDAT):
        actloc[i] = i
    
# Set up the rotation/anisotropy matrices that are needed for the
# variogram and search:

    print('Setting up rotation matrices for variogram and search')
    radsqd = radius * radius
    rotmat = []
    for ic in range(0,ncut):  
        rotmat_temp, maxcov[ic] = setup_rotmat(c0[ic],int(nst[ic]),it[:,ic],cc[:,ic],ang[:,ic],9999.9)
        rotmat.append(rotmat_temp)    
# Initialize accumulators:  # not setup yet
    nk = 0
    xk = 0.0
    vk = 0.0
    for icut in range (0,ncut):
        nviol[icut] =  0
        aviol[icut] =  0.0
        xviol[icut] = -1.0
    nxy   = nx*ny
    print('Working on the kriging')

# Report on progress from time to time:
    if koption == 0: 
        nxy   = nx*ny
        nloop = nxy
        irepo = max(1,min((nxy/10),10000))
    else:
        nloop = 10000000
        irepo = max(1,min((nd/10),10000))
    ddh = 0.0
    
# MAIN LOOP OVER ALL THE BLOCKS IN THE GRID:
    for index in range(0,nloop):
      
        if (int(index/irepo)*irepo) == index: print('   currently on estimate ' + str(index))
    
        if koption == 0:
            iy   = int((index)/nx) 
            ix   = index - (iy)*nx
            xloc = xmn + (ix)*xsiz
            yloc = ymn + (iy)*ysiz
        else:
            ddh = 0.0
            # TODO: pass the cross validation value

# Find the nearest samples within each octant: First initialize the counter arrays:
        na = -1   # accounting for 0 as first index
        dist.fill(1.0e+20)
        nums.fill(-1)
        current_node = (yloc,xloc)
        print('current location ' + str(current_node))
        dist, close = tree.query(current_node,ndmax) # use kd tree for fast nearest data search
        # remove any data outside search radius
        close = close[dist<radius]
        dist = dist[dist<radius]     
        
        print('close' + str(close))
        print('dist' + str(dist))
        
        nclose = len(dist) 

# Is there enough samples?

        if nclose < ndmin:   # accounting for min index of 0
            for i in range(0,ncut):
                ccdfo[i] = UNEST
            print('UNEST at ' + str(ix) + ',' + str(iy))
        else:         

# Loop over all the thresholds/categories:
            for ic in range(0,ncut):
                nclose = len(dist)
                print()
                print('Threshold ' + str(ic))
                krig = True
                if mik == 1 and ic >= 1: krig = False

# Identify the close data (there may be a different number of data at
# each threshold because of constraint intervals); however, if
# there are no constraint intervals then this step can be avoided.
                nca = -1
                for ia in range(0,nclose):
                    j  = int(close[ia]+0.5)
                    ii = actloc[j]
                    accept = True
                    if koption != 0 and (abs(x[j]-xloc) + abs(y[j]-yloc)).lt.EPSLON: 
                        print('offset reject')
                        accept = False
#                     print('Vr ' + str(vr[ii,ic]))
                    if vr[ii,ic] < -9990: # remove null values, soft contraints
                        print('Data Removed ' + str(ii))
                        accept = False
                    if accept:
#                         print('Accepted ' + str(vr[ii,ic]))
                        nca = nca + 1
                        vra[nca] = vr[ii,ic]
                        xa[nca]  = x[j]
                        ya[nca]  = y[j]

                        
                print('nca ' + str (nca))
                print('vra ' +  str (vra))
                print('xa ' + str (xa))
                print('ya ' + str (ya))
# If there are no samples at this threshold then use the global cdf:
                if nca == -1:
                    if use_trend:
                        ccdf[ic] = trend[ny-iy-1,ix,ic]
                    else:
                        ccdf[ic] = gcdf[ic]
                else:
            
# Now, only load the variogram, build the matrix,... if kriging:
                    nclose = nca + 1
                    neq = nclose + ktype
                    na = nclose
                    print('nclose ' + str(nclose))

# Set up kriging matrices:
                    iin=-1 # accounting for first index of 0
                    for j in range(0,na):
# Establish Left Hand Side Covariance Matrix:
                        for i in range(0,na):  # was j - want full matrix                    
                            iin = iin + 1
                            a[iin] = cova2(xa[i],ya[i],xa[j],ya[j],nst[ic],c0[ic],PMX,cc[:,ic],aa[:,ic],it[:,ic],ang[:,ic],anis[:,ic],rotmat[ic],maxcov[ic]) 
                        if ktype == 1:
                            iin = iin + 1
                            a[iin] = maxcov[ic]            
                        r[j] = cova2(xloc,yloc,xa[j],ya[j],nst[ic],c0[ic],PMX,cc[:,ic],aa[:,ic],it[:,ic],ang[:,ic],anis[:,ic],rotmat[ic],maxcov[ic]) 
    
# Set the unbiasedness constraint:
                    if ktype == 1:
                        for i in range(0,na):
                            iin = iin + 1
                            a[iin] = maxcov[ic]
                        iin      = iin + 1
                        a[iin]   = 0.0
                        r[neq-1]  = maxcov[ic]
                        rr[neq-1] = r[neq]
# Solve the system:
                    if neq == 1:
                        ising = 0.0
                        s[0]  = r[0] / a[0]
                    else:
                        s = ksol_numpy(neq,a,r)

# Finished kriging (if it was necessary):

# Compute Kriged estimate of cumulative probability:
                    sumwts   = 0.0
                    ccdf[ic] = 0.0
                    #print('Use trend ' + str(use_trend))
                    for i in range(0,nclose):
                        ccdf[ic] = ccdf[ic] + vra[i]*s[i]
                        sumwts   = sumwts   + s[i]
                        
                    print('Kriging:')    
                    print('sumwts' + str(sumwts))
                    print('nclose ' + str(nclose))
                    print(vra[:nclose])
                    print('Weights ' + str(s[:nclose]))
                    print('Kriging Estimate ' + str(ccdf[ic]))
                    
                    if ktype == 0: 
                        if use_trend == True:
                            ccdf[ic] = ccdf[ic] + (1.0-sumwts)*trend[ny-iy-1,ix,ic]
                        else:
                            ccdf[ic] = ccdf[ic] + (1.0-sumwts)*gcdf[ic]

                    print('Kriging Estimate After Global ' + str(ccdf[ic]))                            
                            
# Keep looping until all the thresholds are estimated:
 
# Correct and write the distribution to the output file:
            print('Before order relations ' + str(ccdf))
            nk = nk + 1
            ccdfo = ordrel(ivtype,ncut,ccdf)
            print('After order relations ' + str(ccdfo))
        
# Write the IK CCDF for this grid node:
            if koption == 0:
                print('iy ' + str(ny-iy-1) + ' ix ' + str(ix))
                print('print shape output ' + str(ikout.shape))
                print('ny' + str(ny) + ' nx ' + str(nx))
                ikout[ny-iy-1,ix,:] = ccdfo
            else:
                 print('TBD')
                    
#             if ix == 170:
#                 asl;ghal;g
    return ikout  


For this example lets just assume the indicator variograms for the 2 facies, 0 is shale and 1 is sand.  We declare a list and then add the variograms in the order of the categories in the thresh and global cdf arrays.

In [ ]:
xmin = 0.0; xmax = 1000.0               # range of x values
ymin = 0.0; ymax = 1000.0               # range of y values
radius = 10000
ivtype = 1
ktype = 0
xsiz = 10; ysiz = 10
nx = 200; ny = 1
xmn = 0; ymn = 1000
xsiz = 10; ysiz = 10
tmin = 10; tmax = 30; ndmin = 0; ndmax = 9

ncut = 11                                   # number of facies
thresh = [10,12,14,16,18,20,22,24,26,28,30]                             # the facies categories
gcdf =   [0.03,0.06,0.15,0.30,0.50,0.65,0.83,0.87,0.91,0.93,0.97]                           # the global proportions of the categories
varios = [] # the variogram list
for i in range(0,12):
    varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=1000,hmin1=1000)) # shale indicator variogram
#varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=50,hmin1=40)) # sand indicator variogram

In [ ]:
readind = True
colstart = 5
ikmap = ik2d(df,'x','y','z',readind,colstart,ivtype,0,ncut,thresh,gcdf,np.zeros([5,5]),tmin,tmax,nx,xmn,xsiz,ny,ymn,ysiz,
         ndmin,ndmax,radius,ktype,vario=varios)
#def ik2d(df,xcol,ycol,vcol,readind,colstart,ivtype,koption,ncut,thresh,gcdf,trend,tmin,tmax,nx,xmn,xsiz,ny,ymn,ysiz,
#         ndmin,ndmax,radius,ktype,vario): 

In [ ]:
cprob = np.zeros((nx,10))
xgrid = np.linspace(xmn,nx*xsiz-xmn,nx)

plt.subplot(211)
for ix, x in enumerate(np.linspace(xmn,nx*xsiz-xmn,nx)):
    for icumprob, cumprob in enumerate(np.linspace(0.05,0.95,10)):
        cprob[ix,icumprob] = np.interp(cumprob,ikmap[0,ix,:],thresh,left=None, right=None, period=None)

xnew = np.linspace(0.0,2000,300); plevels = np.linspace(0.05,0.95,10)
spline_surf = np.zeros([len(xnew),len(plevels)])
        

for icumprob, cumprob in enumerate(plevels):
    #plt.plot(xgrid,cprob[:,icumprob],color='grey',alpha=0.3,zorder=10) 
    spline = make_interp_spline(xgrid,cprob[:,icumprob], k=2)
    alpha = 1-abs(cumprob-0.5); lw = (1-abs(cumprob-0.5)*1.5)*1.5; 
    spline_surf[:,icumprob] = spline(xnew) 
    plt.plot(xnew[1:-1],spline_surf[:,icumprob][1:-1],color='black',alpha=alpha,lw =lw,zorder=2000)
    
plt.fill_between(xnew,spline_surf[:,2],spline_surf[:,3],color='yellow',alpha=0.5,zorder=1,label='P25-P75')
plt.fill_between(xnew,spline_surf[:,3],spline_surf[:,4],color='orange',alpha=0.5,zorder=1,label='P35-P65')
plt.fill_between(xnew,spline_surf[:,4],spline_surf[:,5],color='red',alpha=0.5,zorder=1,label='P45-P55')
plt.fill_between(xnew,spline_surf[:,5],spline_surf[:,6],color='orange',alpha=0.5,zorder=1)
plt.fill_between(xnew,spline_surf[:,6],spline_surf[:,7],color='yellow',alpha=0.5,zorder=1)
plt.legend(loc = 'upper left')
                 
for idata in range(0,len(df)):
    plt.vlines(df['x'][idata],10,30.0,color='black',ls='--',zorder=10000)
    
# plt.legend(loc='upper right')
    
plt.ylim([10,30]); plt.xlim([0,2000]); plt.xlabel('X (m)'); plt.ylabel('Z (m)'); plt.title('Soft Well Data Bounding Surface Constraints - PDFs')

for idata, zmin in enumerate(df['zmin']):
    zmax = df['zmax'][idata]
    if zmin == zmax:
        plt.scatter(df['x'][idata],zmin,c='white',edgecolor='black',s=30,zorder=100000)
    if zmin < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='white',lw=3,zorder=100001)
    if zmax < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='white',lw=3,zorder=100001)
        
add_grid()
        
plt.subplot(212)
for icut in range(0,ncut):
#    print(thresh[icut])
    for ix in range(1,nx-1):
#         print(xgrid[ix],thresh[icut])
        im = plt.scatter(xgrid[ix],thresh[icut],s=30,color = plt.cm.inferno(ikmap[0,ix,icut]),marker='s',cmap = plt.cm.inferno,zorder=100)

for ix, x in enumerate(np.linspace(xmn,nx*xsiz-xmn,nx)):
    for icumprob, cumprob in enumerate(np.linspace(0.05,0.95,10)):
        cprob[ix,icumprob] = np.interp(cumprob,ikmap[0,ix,:],thresh,left=None, right=None, period=None)
    
for idata in range(0,len(df)):
    plt.vlines(df['x'][idata],10,30.0,color='black',ls='--',zorder=10000)
    
delta = 0.3
for ithresh, thr in enumerate(thresh):
    plt.plot([0,2000],[thr-delta,thr-delta],color='black'); plt.plot([0,2000],[thr+delta,thr+delta],color='black') 
    
# plt.legend(loc='upper right')
    
plt.ylim([10,30]); plt.xlim([0,2000]); plt.xlabel('X (m)'); plt.ylabel('Z (m)'); plt.title('Soft Well Data Bounding Surface Constraints - Indicator Kriged Probabilities')

for idata, zmin in enumerate(df['zmin']):
    zmax = df['zmax'][idata]
    if zmin == zmax:
        plt.scatter(df['x'][idata],zmin,c='white',edgecolor='black',s=30,zorder=100000)
    if zmin < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='white',lw=3,zorder=100001)
    if zmax < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='white',lw=3,zorder=100001)

sm = plt.cm.ScalarMappable(cmap=plt.cm.inferno)
sm.set_clim(vmin=0, vmax=1.0)
cbar = plt.colorbar(sm, orientation="horizontal", ticks=np.linspace(0.0, 1.0, 6),shrink=0.2)
cbar.set_label('Cumulative Probability', rotation=0, labelpad=0.1)
add_grid()

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=2.0, wspace=0.1, hspace=0.3); plt.show()

#### Visualize Local Uncertainty Distributions

In [ ]:
ix = [25, 100, 150]

for iix in range(0,3):
    
    plt.subplot(3,1,iix+1)
    
    plt.plot(thresh,ikmap[0,ix[iix],:],color='black',lw=4,zorder=1)
    plt.plot(thresh,ikmap[0,ix[iix],:],color='red',lw=2,zorder=2)
    plt.scatter(thresh,ikmap[0,ix[iix],:],color='white',edgecolor='black',lw=2,zorder=100)
    
    plt.xlim([10,30]); plt.ylim([0,1]); plt.xlabel('Z (m)'); 
    plt.ylabel('Cumulative Probability'); 
    plt.title('CDF Estimated from Soft Data Constraint at X = ' + str(xmn + ix[iix]*xsiz))

    for i in range(0,ncut):
        plt.vlines(thresh[i],0,1,color='black',ls='--',lw=1)

    add_grid()

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=3.0, wspace=0.2, hspace=0.3); plt.show()

### P-field Simulation

Extremely flexible method to calculate stochastic simulation from local PDFs, that separates the correlation and local conditioning. 

* Exceptional ability to honor additional information source

* Very good diagnostics

In [ ]:
# Pfields
nxdis = 1; nydis = 1; nreal = 100
ndmin = 0; ndmax = 20; radius = 10000; skmean = 0
vario = GSLIB.make_variogram(nug=0.001,nst=1,it1=3,cc1=0.999,azi1=90.0,hmaj1=1500,hmin1=1)
tmin = -999; tmax = 999

dfnull = pd.DataFrame({'X':np.random.rand(1000)-100000, 'Y':np.random.rand(1000)-100000,'pvalue':np.random.rand(1000)})

pfield = geostats.sgsim(dfnull,'X','Y','pvalue',wcol=-1,scol=-1,tmin=tmin,tmax=tmax,itrans=1,ismooth=0,dftrans=0,tcol=0,
            twtcol=0,zmin=0.0,zmax=1.0,ltail=1,ltpar=0.0,utail=1,utpar=1.0,nsim=1,
            nx=nx,xmn=xmn,xsiz=xsiz,ny=nreal,ymn=ymn,ysiz=ysiz,seed=73073,
            ndmin=ndmin,ndmax=ndmax,nodmax=50,mults=1,nmult=3,noct=-1,radius=radius,radius1=100000,sang1=0,
            mxctx=201,mxcty=81,ktype=0,colocorr=0.0,sec_map=0,vario=vario)

Let's check out 1D correlated P-fields

In [ ]:
smooth_pfield = np.zeros([nreal,nx])
kernel = np.ones([3])/3.0

for ireal, real in enumerate(np.arange(0,nreal,1)): # smoothing
    smooth_pfield[ireal,:] = np.convolve(pfield[ireal,:],kernel,mode = 'same') 
    
plt.subplot(111)
for ireal, real in enumerate(np.arange(0,10,1)): # smoothing
    for ix in range(1,nx-1):
        im = plt.scatter(xgrid[ix],ireal+1,s=30,color = plt.cm.inferno(smooth_pfield[ireal,ix]),marker='s',cmap = plt.cm.inferno,zorder=100)

delta = 0.13
for ireal, real in enumerate(np.arange(0,10,1)): # smoothing
    plt.plot([0,2000],[ireal-delta,ireal-delta],color='black'); plt.plot([0,2000],[ireal+delta,ireal+delta],color='black') 
         
plt.xlabel('X (m)'); plt.ylabel('Relization (index)')
plt.ylim([1,10]); plt.xlim([0,2000]); plt.xlabel('X (m)'); plt.title('P-feilds - Correlated Probability Fields')

sm = plt.cm.ScalarMappable(cmap=plt.cm.inferno)
sm.set_clim(vmin=0, vmax=1.0)
cbar = plt.colorbar(sm, orientation="horizontal", ticks=np.linspace(0.0, 1.0, 6),shrink=0.2)
cbar.set_label('P-value Realizations', rotation=0, labelpad=0.1)
add_grid()

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.0, wspace=0.2, hspace=0.2); plt.show()

Now we apply our P-fields to our local uncertainty distributions from our hard and soft well data.

In [ ]:
nreal = 10

kernel = np.ones([3])/3.0

sim = np.zeros([nx,nreal]); smooth_sim = np.zeros([nx,nreal])
for ireal, real in enumerate(np.arange(0,nreal,1)):
    for ix, x in enumerate(np.linspace(xmn,nx*xsiz-xmn,nx)):
        sim[ix,ireal] = np.interp(pfield[ireal,ix],ikmap[0,ix,:],thresh,left=None, right=None, period=None)  

for ireal, real in enumerate(np.arange(0,nreal,1)): # smoothing
    smooth_sim[:,ireal] = np.convolve(sim[:,ireal],kernel,mode = 'same') 
        
plt.subplot(2,1,1)
for ireal, real in enumerate(np.arange(0,nreal,1)):
    plt.plot(np.linspace(xmn,nx*xsiz-xmn,nx)[1:-1],pfield[ireal,1:-1],alpha=1.0,
            color = plt.cm.inferno(ireal/(nreal-1)),lw=3,zorder=10)
    plt.plot(np.linspace(xmn,nx*xsiz-xmn,nx)[1:-1],pfield[ireal,1:-1],alpha=1.0,
            color = 'black',lw=4,zorder=1)    
add_grid()
    
plt.ylim([0,1]); plt.xlim([0,2000]); plt.xlabel('X (m)'); plt.ylabel('Z (m)'); plt.title('Stochastic P-fields')

plt.subplot(2,1,2)        
for ireal, real in enumerate(np.arange(0,nreal,1)):
    plt.plot(np.linspace(xmn,nx*xsiz-xmn,nx)[1:-1],smooth_sim[1:-1,ireal],alpha=1.0,
            color = plt.cm.inferno(ireal/(nreal-1)),lw=3,zorder=10)
    plt.plot(np.linspace(xmn,nx*xsiz-xmn,nx)[1:-1],smooth_sim[1:-1,ireal],alpha=1.0,
            color = 'black',lw=4,zorder=1) 

    
for idata in range(0,len(df)):
    plt.vlines(df['x'][idata],10,30.0,color='black',ls='--',zorder=10000)

for idata, zmin in enumerate(df['zmin']):
    zmax = df['zmax'][idata]
    if zmin == zmax:
        plt.scatter(df['x'][idata],zmin,c='white',edgecolor='black',s=30,zorder=100000)
    if zmin < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[30,zmax],color='white',lw=3,zorder=100001)
    if zmax < -9990:
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='black',lw=5,zorder=100000)
        plt.plot([df['x'][idata],df['x'][idata]],[10,zmin],color='white',lw=3,zorder=100001)
add_grid()

plt.ylim([10,30]); plt.xlim([0,2000]); plt.xlabel('X (m)'); plt.ylabel('Z (m)'); plt.title('Soft Well Data Bounding Surface Simulations')

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=2.0, wspace=0.2, hspace=0.5); plt.show()

*Michael*

**Michael Pyrcz**, Ph.D., P.Eng. Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin
On Twitter I'm the **GeostatsGuy** and on YouTube my lectures are on the channel, **GeostatsGuy Lectures**.